In [39]:
import pandas as pd
import numpy as np
import json
pd.set_option("display.max_columns", None)
from sklearn.model_selection import train_test_split

In [40]:
X= pd.read_csv("raw_data/games_V4.csv")
y= pd.read_csv("raw_data/y_rating.csv")

In [6]:
X.columns

Index(['App_ID', 'Name', 'Release_Date', 'Estimated_Owners', 'Peak_CCU',
       'Required_Age', 'Price', 'DLC_Count', 'About_The_Game',
       'Supported_Languages', 'Full_Audio_Languages', 'Reviews',
       'Header_Image', 'Website', 'Support_URL', 'Support_Email', 'Windows',
       'Mac', 'Linux', 'Metacritic_Score', 'Metacritic_URL', 'User_Score',
       'Positive', 'Negative', 'Score_Rank', 'Achievements', 'Recommendations',
       'Notes', 'Average_Playtime_Forever', 'Average_Playtime_2Weeks',
       'Median_Playtime_Forever', 'Median_Playtime_2Weeks', 'Developers',
       'Publishers', 'Categories', 'Genres', 'Tags', 'Screenshots', 'Movies'],
      dtype='object')

In [41]:
images = X[["App_ID","Screenshots"]].copy()

In [42]:
images["Screenshots"].head().loc[0].split(",")

['https://cdn.akamai.steamstatic.com/steam/apps/32400/ss_7ae5b4d31b0e79ea86a33797ca0f552567e18e18.1920x1080.jpg?t=1709124796',
 'https://cdn.akamai.steamstatic.com/steam/apps/32400/ss_4287f2e96631220e2a1184865f52d7da9b62e771.1920x1080.jpg?t=1709124796',
 'https://cdn.akamai.steamstatic.com/steam/apps/32400/ss_f1cc82b9fd820f22cb5715378d8064aab7887153.1920x1080.jpg?t=1709124796',
 'https://cdn.akamai.steamstatic.com/steam/apps/32400/ss_4db138b0790b880a8f0820bff5129ae0d20d9f48.1920x1080.jpg?t=1709124796',
 'https://cdn.akamai.steamstatic.com/steam/apps/32400/ss_7c0b9d994dfeb726500303485cfd141d4df3c5e5.1920x1080.jpg?t=1709124796',
 'https://cdn.akamai.steamstatic.com/steam/apps/32400/ss_b0946a6597b117a70602b9fa400d6273021eb7c8.1920x1080.jpg?t=1709124796',
 'https://cdn.akamai.steamstatic.com/steam/apps/32400/ss_d4debfaf63415c0b67d0283dd43fdbb8983da4dd.1920x1080.jpg?t=1709124796',
 'https://cdn.akamai.steamstatic.com/steam/apps/32400/ss_c58a0a5b008b7c78c558d597dc53bfa1689c7963.1920x1080.jpg

In [43]:
images["Screenshots"].isna().sum()

5

In [37]:
import numpy as np
import os
import pandas as pd
import requests
import os
from io import BytesIO
from PIL import Image

def download_image(url, app_id, index, folder_name='downloaded_images', size=(256, 256)):
    try:
        # Obtenir l'image depuis l'URL
        response = requests.get(url)
        response.raise_for_status()  # Ceci va arrêter le processus en cas d'erreur

        # Convertir le contenu binaire de l'image en un objet Image et redimensionner
        image = Image.open(BytesIO(response.content))
        image = image.resize(size)  # Redimensionner l'image en 256x256 pixels

        # Construire le chemin du fichier
        file_path = f"{folder_name}/{app_id}_{index}.jpg"

        # Créer le dossier s'il n'existe pas
        os.makedirs(folder_name, exist_ok=True)

        # Écrire l'image redimensionnée dans un fichier en ajustant la qualité pour la compression
        image.save(file_path, 'JPEG')  # Réduire la qualité pour compresser l'image
        return file_path
    except Exception as e:
        print(f"Erreur lors du téléchargement de {url}: {e}")
        return None


def save_blank_image(app_id, index, image_size=(256, 256)):
    # Create a blank (all white) image
    blank_image = np.ones((image_size[1], image_size[0], 3), dtype=np.uint8) * 255  # Note the order of dimensions
    blank_image = Image.fromarray(blank_image)
    blank_image.save(f"{app_id}_{index}.png")

for index, row in images.iterrows():
    app_id = row['App_ID']
    # Check if 'Screenshots' exists and is not NA
    if pd.isna(row['Screenshots']):
        # 'Screenshots' is NA, save a blank image
        save_blank_image(app_id, 0)  # Using 0 for index since there's no screenshot
    else:
        # 'Screenshots' is not NA, proceed to split and download each screenshot
        screenshots = row['Screenshots'].split(',')
        for i, screenshot in enumerate(screenshots[:4]):
            download_image(screenshot, app_id, i)

KeyError: 'Screenshots'

In [18]:
import os
from tensorflow.keras.utils import to_categorical
from tqdm import tqdm
import numpy as np
import pandas as pd
from PIL import Image
from tensorflow.keras.preprocessing.image import load_img, img_to_array

dossier_images = "downloaded_images"

chemins_images = []

for fichier in os.listdir(dossier_images):
    # Vérifier si le fichier est une image
    if fichier.endswith(".jpg") or fichier.endswith(".jpeg") or fichier.endswith(".png"):
        # Construire le chemin complet de l'image
        chemin_image = os.path.join(dossier_images, fichier)
        # Ajouter le chemin de l'image à la liste
        chemins_images.append(chemin_image)

# images = []
# for chemin_image in chemins_images:
#     # Charger l'image avec PIL
#     image = load_img(chemin_image)
#     # Ajouter l'image à la liste
#     images.append(image)

In [3]:
chemins_images

['downloaded_images/1558830_29.jpg',
 'downloaded_images/1428420_14.jpg',
 'downloaded_images/1154810_8.jpg',
 'downloaded_images/1559430_4.jpg',
 'downloaded_images/1788780_22.jpg',
 'downloaded_images/332950_4.jpg',
 'downloaded_images/1102720_13.jpg',
 'downloaded_images/1049850_11.jpg',
 'downloaded_images/2233790_4.jpg',
 'downloaded_images/2344520_9.jpg',
 'downloaded_images/1084640_5.jpg',
 'downloaded_images/296490_4.jpg',
 'downloaded_images/327950_3.jpg',
 'downloaded_images/679090_2.jpg',
 'downloaded_images/2529610_0.jpg',
 'downloaded_images/1119570_1.jpg',
 'downloaded_images/2218750_7.jpg',
 'downloaded_images/15190_1.jpg',
 'downloaded_images/445720_6.jpg',
 'downloaded_images/1061680_5.jpg',
 'downloaded_images/544580_5.jpg',
 'downloaded_images/790730_4.jpg',
 'downloaded_images/367550_65.jpg',
 'downloaded_images/315460_0.jpg',
 'downloaded_images/1726760_10.jpg',
 'downloaded_images/1606340_8.jpg',
 'downloaded_images/665490_1.jpg',
 'downloaded_images/1031270_7.jpg

In [19]:
import re
def extract_numeric(filename):
    # Rechercher les parties numériques dans le nom de fichier
    match = re.search(r'\d+', filename)
    if match:
        return match.group()
    else:
        return None
noms_images = [extract_numeric(chemin) for chemin in chemins_images]

In [20]:
df_chemin = pd.DataFrame(noms_images, columns=['App_ID'])

In [21]:
df_chemin = df_chemin.astype(int)

In [35]:
list_App= df_chemin.groupby("App_ID").sum().reset_index()

In [36]:
list_App

,App_ID
0,10
1,20
2,80
3,100
4,220
...,...
9634,2716110
9635,2717080
9636,2718190
9637,2718200


In [30]:
set1 = set(X["App_ID"])
set2 = set(list_App)

# Trouver la différence entre les deux ensembles
difference = set1 - set2

# Convertir la différence en liste, si nécessaire
difference_list = list(difference)

In [32]:
len(difference_list)

6217

0

In [16]:
folder_path = 'downloaded_images'

# Liste pour conserver les noms des fichiers à supprimer
files_to_delete = []

# Parcourir tous les fichiers dans le dossier
for filename in os.listdir(folder_path):
    # Vérifier si le fichier ne se termine pas par '_1.jpg', '_2.jpg', ou '_3.jpg'
    if not (filename.endswith('_0.jpg') or filename.endswith('_1.jpg') or filename.endswith('_2.jpg') or filename.endswith('_3.jpg')):
        # Ajouter le fichier à la liste des fichiers à supprimer
        files_to_delete.append(filename)

# Afficher la liste des fichiers qui seront supprimés
len(files_to_delete)

64071

In [17]:
for filename in files_to_delete:
    # Construire le chemin complet vers le fichier
    file_path = os.path.join(folder_path, filename)
    # Vérifier si le fichier existe avant de tenter de le supprimer
    if os.path.exists(file_path):
        # Supprimer le fichier
        os.remove(file_path)
    else:
        # Le fichier n'existe pas
        print(f'Le fichier {filename} n\'existe pas.')